## Install the package dependencies before running this notebook

In [11]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob

"""
    number of trajectories in each city
    # austin --  train: 43041 test: 6325 
    # miami -- train: 55029 test:7971
    # pittsburgh -- train: 43544 test: 6361
    # dearborn -- train: 24465 test: 3671
    # washington-dc -- train: 25744 test: 3829
    # palo-alto -- train:  11993 test:1686

    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds
    
"""

'\n    number of trajectories in each city\n    # austin --  train: 43041 test: 6325 \n    # miami -- train: 55029 test:7971\n    # pittsburgh -- train: 43544 test: 6361\n    # dearborn -- train: 24465 test: 3671\n    # washington-dc -- train: 25744 test: 3829\n    # palo-alto -- train:  11993 test:1686\n\n    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds\n    \n'

## Create a Torch.Dataset class for the training dataset

In [12]:
from glob import glob
import pickle
import numpy as np

ROOT_PATH = "./"

cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"]
splits = ["train", "test"]

def get_city_trajectories(city="palo-alto", split="train", normalized=False):
    f_in = ROOT_PATH + split + "/" + city + "_inputs"
    inputs = pickle.load(open(f_in, "rb"))
    inputs = np.asarray(inputs)
    
    outputs = None
    
    if split=="train":
        f_out = ROOT_PATH + split + "/" + city + "_outputs"
        outputs = pickle.load(open(f_out, "rb"))
        outputs = np.asarray(outputs)

    return inputs, outputs

def get_training_trajectories(normalized=False):
    """ 
    Get the training trajectories of all cities.
    This is useful for if we wish to ignore the city features. 
    """
    first_iter = True
    cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"]
    for city in cities:
        print(city)
        # Assign initial array
        if first_iter:
            first_iter = False
            inputs, outputs = get_city_trajectories(city, split='train', normalized=normalized)
            continue
        # Get city's trajectories
        city_in, city_out = get_city_trajectories(city, split='train', normalized=normalized)
        #print(city_in.shape, city_out.shape)
        inputs = np.concatenate([inputs, city_in])
        outputs = np.concatenate([outputs, city_out])
        
    #print(inputs.shape)
    #print(outputs.shape)
    return inputs, outputs


def get_test_coords(normalized=False):
    """ Retrieve only the test data, used for submission into the csv set. """
    first_iter = True
    cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"]
    for city in cities:
        # Assign initial array
        if first_iter:
            first_iter = False
            inputs, outputs = get_city_trajectories(city, split='test', normalized=normalized)
            continue
        # Get city's trajectories
        city_in, outputs = get_city_trajectories(city, split='test', normalized=normalized)
        inputs = np.concatenate([inputs, city_in])
    return inputs


class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, city: str, split:str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.transform = transform

        self.inputs, self.outputs = get_city_trajectories(city=city, split=split, normalized=False)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):

        data = (self.inputs[idx], self.outputs[idx])
            
        if self.transform:
            data = self.transform(data)

        return data

class ArgoverseFullDataset(Dataset):
    """ Dataset class for Argoverse. Uses inputs from all cities in contrast to ArgoverseDataset's single city. """
    def __init__(self, transform=None):
        super(ArgoverseFullDataset, self).__init__()
        self.transform = transform

        self.inputs, self.outputs = get_training_trajectories()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):

        data = (self.inputs[idx], self.outputs[idx])
            
        if self.transform:
            data = self.transform(data)

        return data
    
    
# intialize a dataset
city = 'palo-alto' 
split = 'train'
train_dataset  = ArgoverseDataset(city = city, split = split)

In [13]:
train_dataset = ArgoverseFullDataset()

austin
miami
pittsburgh
dearborn
washington-dc
palo-alto


In [14]:
get_city_trajectories()[0].shape

(11993, 50, 2)

In [15]:
get_test_coords().shape

(29843, 50, 2)

## Create a DataLoader class for training

In [108]:
batch_sz = 500  # batch size 
train_loader = DataLoader(train_dataset,batch_size=batch_sz)

## Sample a batch of data and visualize 

In [111]:
import matplotlib.pyplot as plt
import random


def show_sample_batch(sample_batch):
    """visualize the trajectory for a batch of samples"""
    inp, out = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        #axs[i].xaxis.set_ticks([]) # These will now show ticks
        #axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i,:,0], inp[i,:,1])
        axs[i].scatter(out[i,:,0], out[i,:,1])
# This doesn't display well with batch sizes over like 10
'''
ex_inp = 0       
for i_batch, sample_batch in enumerate(train_loader):
    inp, out = sample_batch
    ex_inp = inp
    """
    TODO:
      implement your Deep learning model
      implement training routine
    """
    show_sample_batch(sample_batch)
    break
'''

'\nex_inp = 0       \nfor i_batch, sample_batch in enumerate(train_loader):\n    inp, out = sample_batch\n    ex_inp = inp\n    """\n    TODO:\n      implement your Deep learning model\n      implement training routine\n    """\n    show_sample_batch(sample_batch)\n    break\n'





# MLP design

In this Multilayer perceptron model, I will attempt to use the previous 50 steps to predict the next step 60 times. This contrasts the MLP model that predicts the next 60 steps given 50 steps.

In [112]:
def transformation(data):
    """ 
    This transformation will take a 
    1x50x2 input tensor and 1x60x2 output tensor 
    to create a 60x50x2 input tensor and a 1x60x2 output tensor.
    This is done so that the MLP will be training on only "1 step" predictions
    by using the previous predictions to make a more accurate prediction.
    This should help with the problem that MLP 60 step predictions are not related by any matter.
    """
    inputs = data[0]
    outputs = data[1]
    
    new_inputs = []
    # Create new input tensors
    for i in range(60):
        # Clip portion of the inputs in order to concatenate output coordinates
        input_clip = inputs[i:]
        output_clip = outputs[max(0, i - 50):i]
        

        # Attach the coordinates together
        new_input = np.concatenate([
            input_clip,
            output_clip
        ])
        
        #print(i, new_input.shape)
        new_inputs.append(new_input)
        
    # Output is just the coordinate at given index
    outputs = torch.flatten(torch.tensor(outputs), start_dim = 0, end_dim=0)
    return (torch.tensor(new_inputs), outputs)
    

In [113]:
import torch
from torch import nn
from torch.nn import functional as F

In [114]:
class RNNModel(nn.Module):
    """The RNN model."""
    def __init__(self, rnn_layer, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        # RNN layer for processing input and giving hidden state
        self.rnn = rnn_layer
        # Input size
        self.input_size = self.rnn.input_size
        self.num_hiddens = self.rnn.hidden_size
        self.layers = self.rnn.num_layers
        self.linear = nn.Linear(self.num_hiddens, self.input_size)

    def forward(self, inputs, state):
        """
        Forward pass of inputs to the rnn. 
        Inputs should be an input tensor in the shape 
        (batch_size, sequence_length (50), input_size of each coordinate (2 minimum))
        State should be a tuple containing hidden state and candidate memory
        (num_layers, batch_size, num_hiddens) for both
        """
        X = inputs.to(torch.float32)
        Y, state = self.rnn(X, state)
        # The fully connected layer will first change the shape of `Y` to
        # (`num_steps` * `batch_size`, `num_hiddens`). Its output shape is
        # (`num_steps` * `batch_size`, `vocab_size`).
        output = self.linear(Y.reshape((-1, Y.shape[-1])))
        return output, state

    def begin_state(self, batch_size=1, device=None):
        if not isinstance(self.rnn, nn.LSTM):
            # `nn.GRU` takes a tensor as hidden state
            return  torch.zeros((self.num_directions * self.rnn.num_layers,
                                 batch_size, self.num_hiddens),
                                device=device)
        else:
            # `nn.LSTM` takes a tuple of hidden states, one for hidden state and one for candidate
            # Shape is 10 * * hidden_size 
            return (
                torch.zeros((self.rnn.num_layers, batch_size, self.num_hiddens)),# device=device),
                torch.zeros((self.rnn.num_layers, batch_size, self.num_hiddens))#, device=device)
            )
    
    def get_input_size():
        return self.input_size


In [118]:
import math
def train_epoch_ch8(net, train_iter, loss, updater, device):
    """Train a net within one epoch (defined in Chapter 8)."""
    state = None
    batch_losses = []
    input_size = net.input_size
    batch_count = 0
    for X, Y in train_iter:
        state = None
        l = 0
        batch_size = X.shape[0]
        if state is None:
            # Initialize `state` when either it is the first iteration or
            # using random sampling
            state = net.begin_state(batch_size=batch_size, device=device)
        else:
            if isinstance(net, nn.Module) and not isinstance(state, tuple):
                # `state` is a tensor for `nn.GRU`
                state.detach_()
            else:
                # `state` is a tuple of tensors for `nn.LSTM` and
                # for our custom scratch implementation
                for s in state:
                    s.detach_()
        # Reshape outputs to match prediction
        y = Y.float()#.reshape((-1, 2)).float()#.T.reshape(-1)
        # Transform X to take the first 49 inputs
        last_X = X[:, -1:, : ].float()
        X = X[:, :-1, :].float()
        #print(X.shape)
        #print(Y.shape)
        #print(last_X.shape)
        X, y = X.to(device), y.to(device)
        # Forward pass x to warm up
        y_hat, state = net(X, state)
        
        
        # Get last prediction as it is an output we need
        y_hat, state = net(last_X, state) # Should be form (batch, 1, 2)
        y_hat = y_hat.reshape((batch_size, 1, input_size))
        prev_in = y_hat
        # Build prediction tensor
        for i in range(59):
            # Predict output
            y_hat_new, state = net(prev_in, state)
            y_hat_new = y_hat_new.reshape((batch_size, 1, input_size)) 
            
            # Concatenate to tensor of predictions
            y_hat = torch.cat([y_hat, y_hat_new.reshape((batch_size, 1, input_size))], dim=1)
            #print('y_hat', y_hat.shape)
            #print('y_hat_new', y_hat_new.shape)
            #print('y shape', y.shape)
            # Output becomes input
            prev_in = y_hat_new
            
        # Calculate loss
        l = loss(y_hat.float(), y)
        # Add for stats
        batch_losses.append(l.item())
        
        # Backpropagate the loss
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()
            l.backward()
            #grad_clipping(net, 1)
            updater.step()
        else:
            l.backward()
            #grad_clipping(net, 1)
            # Since the `mean` function has been invoked
            updater(batch_size=1)
        #metric.add(l * y.numel(), y.numel())
        
        batch_count += 1
        if batch_count % 50 == 0:
            print('batch', batch_count, 'loss', l.item())
    return np.mean(batch_losses)

In [119]:
def train_ch8(net, train_iter, vocab, lr, num_epochs, device,
              use_random_iter=False):
    """Train a model (defined in Chapter 8)."""
    loss = nn.MSELoss()
    # Optimizer for sgd
    updater = torch.optim.SGD(net.parameters(), lr)
    # Train and predict
    for epoch in range(num_epochs):
        loss, time = train_epoch_ch8(
            net, train_loader, loss, updater, device, use_random_iter)
        print(f'(train) epoch {epoch + 1}, loss {total_loss}')

In [120]:
rnn_lstm = torch.nn.LSTM(
    input_size=2,
    hidden_size=256,
    num_layers=1,
    batch_first=True
)
net = RNNModel(rnn_lstm)

num_epochs, lr = 500, .001
loss = nn.MSELoss()
# Optimizer for sgd
updater = torch.optim.SGD(net.parameters(), lr)
device = torch.device('cpu')
# Train and predict
loss, time = train_epoch_ch8(net, train_loader, loss, updater, device)
print(f'(train) epoch, loss {total_loss}')

batch 1 loss 2172444.5
batch 2 loss 2104628.5
batch 3 loss 1990670.125
batch 4 loss 1947581.625
batch 5 loss 1795980.25
batch 6 loss 1917227.875
batch 7 loss 1869495.25
batch 8 loss 1660020.875
batch 9 loss 1611281.25
batch 10 loss 1783558.125
batch 11 loss 1620292.0
batch 12 loss 1470810.5
batch 13 loss 169212674048.0
batch 14 loss 126698012672.0
batch 15 loss 607428476928.0
batch 16 loss 578317058048.0
batch 17 loss 547428663296.0
batch 18 loss 562202607616.0
batch 19 loss 542126276608.0
batch 20 loss 538287964160.0
batch 21 loss 494526726144.0
batch 22 loss 416647675904.0
batch 23 loss 202923704320.0
batch 24 loss 415971311616.0
batch 25 loss 391694548992.0
batch 26 loss 371401785344.0
batch 27 loss 348490006528.0
batch 28 loss 312660983808.0
batch 29 loss 117239472128.0
batch 30 loss 103816765440.0
batch 31 loss 103392288768.0
batch 32 loss 110225268736.0
batch 33 loss 78811488256.0
batch 34 loss 83832733696.0
batch 35 loss 44767215616.0
batch 36 loss 48355397632.0
batch 37 loss 42

KeyboardInterrupt: 

In [ ]:
# X follows the form of (batch, sequence length, then inputs)
batch_sz = 10
sequence_len = 50 # Always first 50 inputs
input_size = 2 # x and y, may contain other features later on
X = torch.rand((batch_sz, sequence_len, input_size))

#rnn_lstm(X, net.begin_state(batch_sz))

In [27]:
sample = torch.rand((4, 2))
sample

tensor([[0.3779, 0.8248],
        [0.7628, 0.0366],
        [0.3324, 0.1314],
        [0.8373, 0.5321]])

In [35]:
sample2 = sample + 1
sample2

tensor([[1.3779, 1.8248],
        [1.7628, 1.0366],
        [1.3324, 1.1314],
        [1.8373, 1.5321]])

In [41]:
torch.concat([sample.reshape((1, 4, 2)), sample2.reshape(1, 4, 2)]).reshape(-1)

tensor([0.3779, 0.8248, 0.7628, 0.0366, 0.3324, 0.1314, 0.8373, 0.5321, 1.3779,
        1.8248, 1.7628, 1.0366, 1.3324, 1.1314, 1.8373, 1.5321])